#  DDP

### 1. running on our DGX
- docker run -it --rm --gpus '"device=4,5,6,7"' -p 12344:12344 --name msalehjahromi --shm-size=192G  --user $(id -u):$(id -g) --group-add 1944259512 --cpuset-cpus=49-96 -v /rsrch7/home/ip_rsrch/wulab/:/rsrch7/home/ip_rsrch/wulab -v /rsrch1/ip/msalehjahromi/:/rsrch1/ip/msalehjahromi --name mori_jupyter nnunetv2:msalehjahromi
- cd /rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100
- jupyter notebook --ip 0.0.0.0 --port 12344
- http://1mcprddgx05

In [1]:
!gpustat

2a974e97ed31              Wed May 28 23:46:10 2025  535.216.01
[0] NVIDIA A100-SXM4-40GB | 27'C,   0 % |   628 / 40960 MB |
[1] NVIDIA A100-SXM4-40GB | 27'C,   0 % |   626 / 40960 MB |
[2] NVIDIA A100-SXM4-40GB | 56'C, 100 % |  4287 / 40960 MB |
[3] NVIDIA A100-SXM4-40GB | 50'C,  83 % |  4845 / 40960 MB |


In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import torch
print(torch.cuda.device_count())   # → 2
print(torch.cuda.current_device()) # → 0 (this maps to your original GPU 2)


2
0


### 2. Probably need installation, restart after and run 3

In [4]:
# ## !pip uninstall -q histolab -y
# # Install required packages first
# import os
# import subprocess
# pip_commands = [
#     ["pip", "install","-q", "--extra-index-url", "https://download.pytorch.org/whl/cu117", 
#      "torch==2.0.0", "torchvision==0.15.0", "omegaconf", "torchmetrics==0.10.3", 
#      "fvcore", "iopath", "xformers==0.0.18", "submitit", "numpy<2.0"],
#     ["pip", "install", "-q",  "--extra-index-url", "https://pypi.nvidia.com", "cuml-cu11"],
#     ["pip", "install","-q",  "black==22.6.0", "flake8==5.0.4", "pylint==2.15.0"],
#     ["pip", "install", "-q", "mmsegmentation==0.27.0"],
#     ["pip", "install","-q", "mmcv-full==1.5.0"],
#     ["pip", "install","-q", "nibabel"]
# ]

# for cmd in pip_commands:
#     try:
#         print(cmd)
#         subprocess.run(cmd, check=True)
#     except subprocess.CalledProcessError as e:
#         print(f"Failed to install packages with command: {cmd}")
#         print(f"Error: {e}")

### 3. running, notice important hyperparams
-  which model are use so vmin -1000., vmax +150, eps 0.00005
- `num-workers`
- `epochs`: 100
- `accum-steps`: 2000 ?
- `max-chunks`: 66 
- `lr`:  lr*0.1 on base ?
- `warmup-steps`: 5k
- `print-every` : 5000
- `val-every` : 40000

In [ ]:
## Install required packages first

import os
import subprocess
# Set required environment variables
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "2"
os.environ["LOCAL_RANK"] = "0"
os.environ["LOCAL_WORLD_SIZE"] = "2"

# Build the command with --install-packages flag removed
command = [
    "python3",
    "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100_v2/2_run_fineTune_ddp_launcher.py", ##changed
    "--num-gpus", "2",
    "--csv", "/rsrch1/ip/msalehjahromi/codes/FineTune/nlst_event_train_val_.csv",
    "--accum-steps", "200", ###############
    "--num-workers", "5",
    "--epochs", "200", ###############
    "--lr", "0.001", ###############
    "--weight-decay", "0.0001",
    "--optimizer", "adamw",
    "--num-attn-heads", "3",
    "--num-layers", "2",
    "--dropout", "0.35",
    "--unfreeze-strategy", "all",  
    "--output", "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/n_GPU_A100_v2/output_ddp_p16_23_1150_ac200_",
    "--print-every", "1000", ################ 2000
    "--val-every", "70000", ################ 50k
    "--metrics-dir", "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/metrics_multi_gpu"
]

# Run the command
try:
    subprocess.run(command, check=True)
except subprocess.CalledProcessError as e:
    print(f"Error occurred: {e}")
    print(f"Command output: {e.output if hasattr(e, 'output') else 'No output available'}")

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warning

Files in the directory: ['n_GPU_A100_v2', '._.gitignore', '._.DS_Store', '._See_attention.ipynb', '1_GPU_H100', 'See_attention.ipynb', 'n_GPU_A100', 'metrics_single_gpu', 'n_H100onH100', '.DS_Store', '1_GPU_A100', 'ReadMe.md', '.gitignore', 'metrics_multi_gpu', '.git']
Files in the directory: ['n_GPU_A100_v2', '._.gitignore', '._.DS_Store', '._See_attention.ipynb', '1_GPU_H100', 'See_attention.ipynb', 'n_GPU_A100', 'metrics_single_gpu', 'n_H100onH100', '.DS_Store', '1_GPU_A100', 'ReadMe.md', '.gitignore', 'metrics_multi_gpu', '.git']


2025-05-28 23:55:59,098 - INFO - Starting training on 2 GPUs with full model copies (DDP)


Train dataset created on rank 
Train dataset created on rank 

==== Dataset Statistics ====
Total samples: 43852
Training samples: 38382
Validation samples: 5470

1-year-cancer:
  Train positive: 1428 (3.72%)
  Val positive: 188 (3.44%)

2-year-cancer:
  Train positive: 1891 (4.93%)
  Val positive: 256 (4.68%)

3-year-cancer:
  Train positive: 2282 (5.95%)
  Val positive: 315 (5.76%)

4-year-cancer:
  Train positive: 2586 (6.74%)
  Val positive: 363 (6.64%)

5-year-cancer:
  Train positive: 2838 (7.39%)
  Val positive: 403 (7.37%)

6-year-cancer:
  Train positive: 2988 (7.78%)
  Val positive: 429 (7.84%)

Total validation samples: 5470
Created distributed validation loader, each rank processes ~2568 samples


2025-05-28 23:56:01,871 - INFO - using MLP layer as FFN


...

2025-05-29 13:11:32,346 - INFO - iteration: 32000 | epoch: 0 | loss: 0.1595 | lr: 0.0000 | type: training | acc_task0: 0.5833 | sens_task0: 0.3315 | spec_task0: 0.8369 | acc_task1: 0.5878 | sens_task1: 0.4183 | spec_task1: 0.7657 | acc_task2: 0.5959 | sens_task2: 0.4851 | spec_task2: 0.7176 | acc_task3: 0.5814 | sens_task3: 0.5539 | spec_task3: 0.6132 | acc_task4: 0.6514 | sens_task4: 0.8792 | spec_task4: 0.2905 | acc_task5: 0.7533 | sens_task5: 0.9862 | spec_task5: 0.0725



Step 33000 (Epoch 1):
Loss: 0.6333
Learning rate: 0.000043
Accuracy:     T0:0.59  T1:0.59  T2:0.60  T3:0.59  T4:0.65  T5:0.75  
Sensitivity:  T0:0.34  T1:0.43  T2:0.49  T3:0.56  T4:0.88  T5:0.98  
Specificity:  T0:0.83  T1:0.76  T2:0.72  T3:0.61  T4:0.30  T5:0.08  


2025-05-29 13:36:16,548 - INFO - iteration: 33000 | epoch: 0 | loss: 0.6333 | lr: 0.0000 | type: training | acc_task0: 0.5862 | sens_task0: 0.3400 | spec_task0: 0.8343 | acc_task1: 0.5908 | sens_task1: 0.4261 | spec_task1: 0.7636 | acc_task2: 0.5991 | sens_task2: 0.4920 | spec_task2: 0.7166 | acc_task3: 0.5850 | sens_task3: 0.5592 | spec_task3: 0.6149 | acc_task4: 0.6534 | sens_task4: 0.8770 | spec_task4: 0.2993 | acc_task5: 0.7542 | sens_task5: 0.9848 | spec_task5: 0.0815



Step 34000 (Epoch 1):
Loss: 0.9388
Learning rate: 0.000043
Accuracy:     T0:0.59  T1:0.60  T2:0.60  T3:0.59  T4:0.66  T5:0.76  
Sensitivity:  T0:0.35  T1:0.43  T2:0.50  T3:0.57  T4:0.88  T5:0.98  
Specificity:  T0:0.83  T1:0.76  T2:0.72  T3:0.62  T4:0.31  T5:0.09  


2025-05-29 14:02:17,524 - INFO - iteration: 34000 | epoch: 0 | loss: 0.9388 | lr: 0.0000 | type: training | acc_task0: 0.5911 | sens_task0: 0.3507 | spec_task0: 0.8332 | acc_task1: 0.5952 | sens_task1: 0.4345 | spec_task1: 0.7638 | acc_task2: 0.6033 | sens_task2: 0.4987 | spec_task2: 0.7180 | acc_task3: 0.5899 | sens_task3: 0.5652 | spec_task3: 0.6184 | acc_task4: 0.6569 | sens_task4: 0.8763 | spec_task4: 0.3093 | acc_task5: 0.7562 | sens_task5: 0.9836 | spec_task5: 0.0925
